In [2]:
import pandas as pd
import random as rd
import numpy as np
import time
from random import randint
from numpy.linalg import matrix_rank

##вычисляем индекс в массиве для cijk
def index_cshki(i, j, k):
    ind = 0
    if i == 1:
        ind+=k+6*(j-2)
    elif i == 2: 
        ind+=30+k+6*(j-3)
    elif i == 3: 
        ind+=54+k+6*(j-4)
    elif i == 4: 
        ind+=72+k+6*(j-5)
    elif i == 5: 
        ind+=84+k
    return ind-1

##считаем новую структурную константу с индексами i,j,k
def new_cijk(i, j, k, zam):
    global orig
    cijk = 0
    for l in range(1, 6):
        for m in range(l+1, 7):
            cijk = cijk+zam[6*(i-1)+l-1]*zam[6*(j-1)+m-1]*orig[index_cshki(l,m,k)+2]
    return cijk

now = time.time()

##считываем табличку из файла
df = pd.read_excel('a61.xls')

##создаём массив структурных констант алгебры в известном нам виде
orig = pd.Series(df.iloc[0])
alg_id = orig[0]
jk_inv = orig[1]

while time.time() < now + 600:
##генерируем матрицу замены и проверяем чтобы векторы были линейно независимы
    zamena1 = [0 for i in range(6)]
    zamena2 = [0 for i in range(6)]
    zamena3 = [0 for i in range(6)]
    zamena4 = [0 for i in range(6)]
    zamena5 = [0 for i in range(6)]
    zamena6 = [0 for i in range(6)]
    while matrix_rank([zamena1,zamena2,zamena3,zamena4,zamena5,zamena6]) < 6:
        zamena1 = [randint(1, 2) for i in range(6)]
        zamena2 = [randint(1, 2) for i in range(6)]
        zamena3 = [randint(1, 2) for i in range(6)]
        zamena4 = [randint(1, 2) for i in range(6)]
        zamena5 = [randint(1, 2) for i in range(6)]
        zamena6 = [randint(1, 2) for i in range(6)]
        zamena = np.concatenate([zamena1, zamena2, zamena3, zamena4, zamena5, zamena6])

##создаём массив структурных констант в новом базисе (для нужного размера берём за основу старый)
    new = orig.copy()

##заполняем новый массив при помощи определённой ранее функции
    for ii in range (1, 6):
        for jj in range (ii+1, 7):
            for kk in range(1, 7):
                new[index_cshki(ii,jj,kk)+2] = new_cijk(ii, jj, kk, zamena)

##вносим новую запись в таблицу
    new[1] = jk_inv
    new[0] = alg_id
    new = new.to_frame().T
    df = pd.concat([df,new],ignore_index=True,axis=0)
df.drop_duplicates()

##записываем новую таблицу обратно в файл
df.to_excel('a61_new.xls')